## <span style='font-family:Georgia'> Objectives
The purpose of this notebook is creating a baseline rule-model, which is to be a benchmark of the neural model developed in the later phase of the project.

In [12]:
# loading packages
import os
import sys
import pandas as pd
import seaborn as sns
import copy
import matplotlib.pyplot as plt

sns.set(style="darkgrid")
from tqdm import tqdm

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from supportive_functions import rm_consecutive_spaces

import nltk
import nltk.data
from polish_sentence_nltk_tokenizer import sentence_tokenizer

import warnings

warnings.filterwarnings(action="ignore")
pd.options.display.max_columns = None

%matplotlib inline
plt.rcParams['figure.figsize'] = (9, 6)

sns.set(
    rc={
        "figure.figsize": (14, 8.27),
        "axes.facecolor": "white",
        "axes.grid": True,
        "grid.color": ".9",
    }
)

In [13]:
def read_files(dir_path: str, sep: str = ",") -> pd.DataFrame:
    """
    Read the files with timestamps from a given catalog.

    Args:
        dir_path (str): Path to the catalog with the files
        sep (str, optional): Separator used in pd.read_csv() function. Defaults to ",".

    Returns:
        pd.DataFrame: Data from all the files concatenated into one dataframe.
    """
    files = os.listdir(dir_path)
    data = pd.DataFrame(
        [], columns=["index", "timestamp_start", "timestamp_stop", "word"]
    )
    for file in tqdm(files):
        f = open(os.path.join(dir_path, file), encoding="utf-8", mode="r")
        name = file.split(".")[0]
        df = pd.read_csv(f, header=None, sep=sep, encoding="utf-8")
        df["index"] = name
        if sep != ",":
            df["timestamp_start"] = df.iloc[:, 0].str.split(",").str[0].str[1:]
            df["timestamp_stop"] = (
                df.iloc[:, 0].str.split(",").str[1].str.split("\)").str[0]
            )
            df["word"] = df.iloc[:, 0].str.split("\)\s").str[1]
            df.drop([0], axis=1, inplace=True)
            df.drop(df.tail(1).index, inplace=True)
        else:
            df["timestamp_start"] = df.iloc[:, 0].str[1:]
            df["timestamp_stop"] = df.iloc[:, 1].str.split("\) \s").str[0]
            df["word"] = df.iloc[:, 0].str.split("\)\s").str[1]

            df.drop([0, 1], axis=1, inplace=True)
            df.drop(df.tail(1).index, inplace=True)
        data = pd.concat([data, df])
        f.close()
    return data

In [14]:
def calculate_pauses(data: pd.DataFrame) -> pd.DataFrame:
    """
    Generate new features describing pauses between two timestamps.

    Args:
        data (pd.DataFrame): Dataframe with preprocessed timestamp
            data separated into 'timestamp_start' and 'timestamp_stop' columns.

    Returns:
        pd.DataFrame: Input dataframe with two new features added,
            describing the pause between the word and after it.
    """
    data.timestamp_start = data.timestamp_start.astype(int)
    data.timestamp_stop = data.timestamp_stop.astype(int)
    data_shifted = pd.concat(
        [data, data.timestamp_start.shift(-1), data.timestamp_stop.shift()], axis=1
    )
    data_shifted.columns = [
        "index",
        "timestamp_start",
        "timestamp_stop",
        "word",
        "timestamp_start_lead",
        "timestamp_stop_lag",
    ]
    data_shifted["pause_before"] = (
        data_shifted.timestamp_start - data_shifted.timestamp_stop_lag
    )
    data_shifted["pause_after"] = (
        data_shifted.timestamp_start_lead - data_shifted.timestamp_stop
    )
    data_shifted.drop(
        ["timestamp_start_lead", "timestamp_stop_lag"], axis=1, inplace=True
    )
    # to discuss
    data_shifted.pause_before.fillna(data_shifted.timestamp_start, inplace=True)
    data_shifted.pause_after.fillna(0, inplace=True)
    data_shifted.reset_index(drop=True, inplace=True)
    for i, row in data_shifted.iterrows():
        if (i > 0) and (
            data_shifted.loc[i, "index"] != data_shifted.loc[i - 1, "index"]
        ):
            data_shifted.loc[i, "pause_before"] = data_shifted.loc[i, "timestamp_start"]
            data_shifted.loc[i - 1, "pause_after"] = 0

    return data_shifted

## <span style='font-family:Georgia'> Implementation of rules

In [15]:
def implement_rules():
    """
    Implement the rules for baseline model.

    Returns:
        Consecutive rules for baseline model.
    """
    
    # tokenizer for separation into sentences
    tokenizer = nltk.data.load('tokenizers/punkt/polish.pickle')

    # extracting list of abbreviations
    abbreviations = sentence_tokenizer._params.abbrev_types
    abbreviations_with_period = [x for x in abbreviations if '.' in x]

    # separating abbreviations with periods inside them
    spaced_abbreviations_with_period = []
    for abbr in abbreviations_with_period:
        if "." in abbr:
            spaced_abbreviations_with_period.append(abbr.replace(".", " "))

    punctuation = list(sentence_tokenizer.PUNCTUATION)
    punctuation.append('-')
    punctuation.remove(';')

    # reading dict generated from morfeusz (all items from train dataset)
    morfeusz_train = pd.read_csv(
        "../../data/outputs/feature_engineering/morfeusz/items_dict.csv"
    )

    # list of conjuctions from morfeusz
    conj_comp_list = morfeusz_train[
        morfeusz_train.grammatical_class.str.contains("comp|conj")
    ].item.tolist()

    # all conjuctions below are from conj_comp_list
    # not_conj - we do not put coma before them
    # yes_conj - we do put coma before them

    not_conj = [
        "a",
        "albo",
        "ali",
        "ani",
        "aniżeli",
        "aż",
        "bowiem",
        "bylem",
        "bym",
        "byśmy",
        "bądź",
        "co",
        "com",
        "coś",
        "czyli",
        "czym",
        "ewentualnie",
        "gdybyś",
        "gdybyśmy",
        "i",
        "inaczej",
        "jednak",
        "kiedyś",
        "lub",
        "miast",
        "miasto",
        "natomiast",
        "ni",
        "niby",
        "niczym",
        "nie",
        "nim",
        "niż",
        "niżem",
        "oraz",
        "tedy",
        "to",
        "tom",
        "toteż",
        "tudzież",
        "tylko",
        "tym",
        "tymczasem",
        "zamiast",
        "zarówno",
        "zatem",
        "zaś",
    ]
    yes_conj = [
        "aby",
        "abyśmy",
        "acz",
        "aczkolwiek",
        "bo",
        "by",
        "chociaż",
        "chociażby",
        "choć",
        "choćby",
        "czy",
        "dopóki",
        "gdy",
        "gdyby",
        "gdyż",
        "im",
        "iż",
        "jak",
        "jakby",
        "jako",
        "jakoby",
        "jeśli",
        "jeżeli",
        "kiedy",
        "ledwo",
        "ponieważ",
        "póki",
        "skoro",
        "więc",
        "zaledwie",
        "zanim",
        "że",
        "żeby",
        "żebyś",
        "żeś",
        "ale",
        "lecz",
    ]

    # list of który która które
    ktory_list = morfeusz_train[morfeusz_train.item.str.startswith("któr")][
        ~morfeusz_train.item.str.contains("ś")
    ].item.tolist()
    ktory_list.remove("którykolwiek")
    ktory_list.remove("którędy")

    #additional list of conjunctions with coma which were not in morfeusz
    additional_list=['dlatego']

    #merge lists of conjuctions
    con_list_yes = yes_conj + ktory_list + additional_list

    return abbreviations, con_list_yes

In [16]:
def symbols_and_noisy_words(which_set):
    # load list of symbols to replace
    symbols_to_replace_infile = open(
        "../../data/outputs/eda/symbols_to_replace.txt", "r", encoding="utf-8"
    )
    symbols_to_replace = symbols_to_replace_infile.read().splitlines()
    # load list of noisy words, i.e. words with letters from outside the Polish alphabet
    noisy_words_infile = open(
        "../../data/outputs/eda/"+which_set+"/noisy_words.txt", "r", encoding="utf-8"
    )
    noisy_words = noisy_words_infile.read().splitlines()
    # load list of letters from outside the Polish alphabet
    non_polish_letters_infile = open(
        "../../data/outputs/eda/"+which_set+"/non_polish_letters.txt", "r", encoding="utf-8"
    )
    non_polish_letters = non_polish_letters_infile.read().splitlines()
    # merge noisy data into one list
    noisy_word=copy.deepcopy(noisy_words)
    noisy_word.extend(non_polish_letters)
    # merge all data into one list
    symbols_to_replace.extend(noisy_words)
    symbols_to_replace.append('+')
    symbols_to_replace.extend(non_polish_letters)
    return symbols_to_replace, noisy_word

In [23]:
class BaselineModel:
    def __init__(self, which_set):
        self.which_set = which_set
        self.symbols_to_replace, self.noisy_word = symbols_and_noisy_words(which_set)
        self.abbreviations, self.con_list_yes = implement_rules()
        self.fullstop_threshold= 0.3 * 1000  # value based on the eda (0.3 seconds converted to miliseconds)
    
    def predict(self):
        # load data
        if self.which_set=='test':
            data = read_files('../../data/source/poleval_fa.validation/validation', sep="\t")
        elif self.which_set=='train':
            data = read_files('../../data/source/poleval_fa.train/train', sep="\t")

        data_calc=calculate_pauses(data).drop(
            ["timestamp_start", "timestamp_stop"], axis=1
        )

        data_calc_joined = (data_calc[["index", "word"]]
        .groupby(["index"])["word"]
        .agg(" ".join)
        .reset_index()
        )

        #replacing symbols
        if self.which_set=='train':
            for symb in self.symbols_to_replace:
                data_calc_joined["word"] = data_calc_joined["word"].apply(
                    lambda x: x.replace(symb, "")
                )

            data_calc_joined["word"] = data_calc_joined["word"].apply(
                rm_consecutive_spaces
            )
            data_calc_joined.columns = ["FileId", "FixedOutput"]
            data = copy.deepcopy(data_calc_joined)

        elif self.which_set=='test':
            for symb in self.noisy_word:
                data_calc_joined['word'] = data_calc_joined['word'
                        ].apply(lambda x: x.replace(symb, ''))

            data_calc_joined['word'] = data_calc_joined['word'
                    ].apply(rm_consecutive_spaces)

            for symb in self.symbols_to_replace:
                data_calc_joined['word'] = data_calc_joined['word'
                        ].apply(lambda x: x.replace(symb, ''))

            data_calc_joined['word'] = data_calc_joined['word'
                    ].apply(rm_consecutive_spaces)
            data_calc_joined.columns = ['FileId', 'FixedOutput']
            data = copy.deepcopy(data_calc_joined)            


        # inserting fullstops after abbreviations + removing doubled spaces
        for abbr in self.abbreviations:
            data['FixedOutput'] = data['FixedOutput'
                    ].apply(lambda x: x.replace(' ' + abbr + ' ', ' '
                            + abbr.replace(' ', ' . ') + ' . '))
        data['FixedOutput'] = data['FixedOutput'].apply(lambda x: \
                x.replace('. .', '.'))

        # inserting commas according to polish language rules
        for conj in self.con_list_yes:
            data['FixedOutput'] = data['FixedOutput'
                    ].apply(lambda x: x.replace(' ' + conj + ' ', ' , ' + conj
                            + ' '))
        data['FixedOutput'] = data['FixedOutput'].apply(lambda x: \
                x.replace('. ,', ','))

        #saving output to file

        # if self.which_set=='train':
        #     data.to_csv(
        #         '../../data/outputs/baseline/including_pauses/train_data.csv',
        #         index=False
        #     )
        # elif self.which_set=='test':
        #     data.to_csv(
        #         "../../data/outputs/baseline/including_pauses/test_data.csv", index=False
        #     )

        return data

### <span style='font-family:Georgia'> Train data

In [24]:
output_train=BaselineModel('train').predict()

100%|██████████| 793/793 [00:15<00:00, 52.26it/s]


In [25]:
output_train.loc[0, 'FixedOutput']

'we wrocławiu walkę ze szkodnikiem rozpoczyna zespół szkół budowlanych przy ul . grabiszyńskiej do akcji na ratunek kasztanowcom zsb zachęcił pobliskie szkoły gimnazjum nr . 6 przy al . pracy sp . nr . 82 przy ul . blacharskiej sp . nr . 109 przy ul . inżynierskiej sp . nr . 15 przy ul . solskiego gimnazjum nr . 40 przy ul . morelowskiego pomoc ma polegać na szukaniu sponsorów , którzy sfinansują specjalistyczne szczepienia koszt szczepienia jednego drzewa to ok . 70 zł vat sponsor , który uratuje kasztanowca ma zapewnioną na nim tabliczkę reklamową podobna akcja odbyła się w świdnicy gdzie szczepionkę dla 99 drzew zasponsorowała firma imp comfort , która kupiła 10 ha . gruntów w mieście i zamierza wybudować tam swoją fabrykę szrotówek kasztanowcowiaczek już od jakiegoś czasu niszczy kasztanowce do niedawna uważano , że tego szkodnika można wytępić tylko poprzez palenie i kompostowanie liści kasztanowca teraz wiemy , że możemy je uratować poprzez specjalne szczepionki , jeśli możesz po

### <span style='font-family:Georgia'> Test data

In [26]:
output_test=BaselineModel('test').predict()

100%|██████████| 200/200 [00:03<00:00, 58.26it/s]


In [27]:
output_test.loc[1, 'FixedOutput']

'prezydent usa george bush powiedział , że odnowa nowego orleanu i innych zalanych terenów potrwa lata stan luizjana wraz ze stolicą nowym orleanem ucierpiał w wyniku ataku huraganu katrina prezydent skrócił swoje wakacje w teksasie oraz zwołał posiedzenie gabinetu , który obejmie kontrolę nad nadzorem akcji ratunkowej jednej z najgorszych katastrof naturalnych bush wyznaczył trzy priorytety dotyczące walki ze skutkami kataklizmu ratowanie życia ofiarom w tym poszukiwanie zaginionych pomoc poszkodowanym i odnalezionym odbudowa i usuwanie szkód w ramach akcji ratunkowej przydzielono dodatkowo 10 tysięcy żołnierzy , którzy wspomogą operację w regionach najbardziej dotkniętych katastrofą zaliczają się do nich części stanów luizjany i mississippi służby medyczne zaoferowały ponad 10 tysięcy łóżek zaś armia amerykańska wysyła swoje helikoptery i łodzie , aby uratować mieszkańców znajdujących się w schronieniu późnym wieczorem w środę rozpoczęła się ewakuacja reszty mieszkańców nowego orlean